In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import networkx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from bokeh.io import output_notebook, show, save

In [ ]:
# Load dataframe
path = '/content/drive/Shareddrives/CMPE295-TeamEquality/RQAR Code/2. Data Cleaning/data/full_QA_pair.csv'
df = pd.read_csv(path)
df

,question_id,question,question_vocab,comment_id,comment_upvotes,suggested_subreddits
0,eib8be,Looking for a subreddit about redundancy (sort...,looking redundancy sort place good idealess wo...,fcp3r91,2,r/antiwork
1,eibax6,ideas for playing/teaching toddlers,idea teaching toddler 3 year old brother alway...,fcq4bew,1,r/kidscrafts
2,eibax6,ideas for playing/teaching toddlers,idea teaching toddler 3 year old brother alway...,fcq4bew,1,r/toddlers
3,eibax6,ideas for playing/teaching toddlers,idea teaching toddler 3 year old brother alway...,fcq4bew,1,r/kidsafevideos
4,eibbys,A subreddit where people tell stories in the c...,tell story comment word looking kind r instead...,fcq40v6,1,r/onewordeach
...,...,...,...,...,...,...
101577,o0oakk,A subreddit to make actual online friends?,make actual friend title,h1wc8ld,2,r/penpals
101578,o0oakk,A subreddit to make actual online friends?,make actual friend title,h1xrbe6,1,r/makenewfriendshere
101579,o0ox9h,What sub for questions about terms and conditions,question term condition video make directed di...,h21ibzt,1,r/legaladviceofftopic
101580,o0pibq,A sub where people can draw plans for my garden,draw plan garden landscape rubbish thinking id...,h1xwdcc,2,r/howto


In [ ]:
df.groupby('suggested_subreddits')['question_id'].count().reset_index().sort_values(by="question_id", ascending=False)

,suggested_subreddits,question_id
15536,r/tipofmytongue,1857
7293,r/helpmefind,1337
2806,r/casualconversation,895
10890,r/nostupidquestions,757
7853,r/ifyoulikeblank,654
...,...,...
7442,r/holdmybanana,1
7443,r/holdmybeaker,1
7445,r/holdmybraincells,1
7446,r/holdmybreedingtube,1


In [27]:
df_pair = df[['suggested_subreddits', 'question_id']].merge(df[['suggested_subreddits', 'question_id']], on='question_id')
df_pair = df_pair[df_pair['suggested_subreddits_x'] != df_pair['suggested_subreddits_y']]
df_cooccurrence = df_pair.groupby(['suggested_subreddits_x', 'suggested_subreddits_y'])['question_id'].nunique().reset_index()
df_cooccurrence[df_cooccurrence['question_id'] > 50].sort_values(by="question_id", ascending=False)

,suggested_subreddits_x,suggested_subreddits_y,question_id
258215,r/helpmefind,r/tipofmytongue,339
535305,r/tipofmytongue,r/helpmefind,339
204265,r/findareddit,r/remindmebot,257
454837,r/remindmebot,r/findareddit,257
535395,r/tipofmytongue,r/namethatsong,195
...,...,...,...
217756,r/frankreich,r/moedansvravie,1
217755,r/frankreich,r/metametaquebec,1
217754,r/frankreich,r/metabarnak,1
217753,r/frankreich,r/memetresmobile,1


In [28]:
output_notebook()

In [29]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4

In [30]:
min_edge_weight = 10
edge_list = df_cooccurrence[df_cooccurrence['question_id'] > min_edge_weight]
scaler = MinMaxScaler(feature_range=(1, 15))
edge_list['weight'] = scaler.fit_transform(edge_list[['question_id']])
G = networkx.from_pandas_edgelist(edge_list, 'suggested_subreddits_x', 'suggested_subreddits_y', 'weight')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [31]:
#Choose a title!
title = 'Subreddit Cooccurrence Graph'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Subreddit", "@index")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node size and color
network_graph.node_renderer.glyph = Circle(
    size=15,
    fill_color='skyblue'
)
network_graph.node_renderer.selection_glyph = Circle(
    size=20,
    fill_color=Spectral4[2]
)
network_graph.node_renderer.hover_glyph = Circle(
    size=25,
    fill_color=Spectral4[1]
)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(
    line_color="#CCCCCC",
    line_alpha=0.5,
    line_width=5
)
network_graph.edge_renderer.selection_glyph = MultiLine(
    line_color=Spectral4[2],
    line_width=5
)
network_graph.edge_renderer.hover_glyph = MultiLine(
    line_color=Spectral4[1],
    line_width=8
)

network_graph.edge_renderer.data_source.data["line_width"] = [G.get_edge_data(a,b)['weight'] for a, b in G.edges()]
network_graph.edge_renderer.glyph.line_width = {'field': 'line_width'}

#Add network graph to the plot
plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

Output hidden; open in https://colab.research.google.com to view.

In [32]:
networkx.write_gpickle(G, "/content/drive/Shareddrives/CMPE295-TeamEquality/RQAR Code/2. Data Cleaning/kg.gpickle")

In [33]:
G = networkx.read_gpickle("/content/drive/Shareddrives/CMPE295-TeamEquality/RQAR Code/2. Data Cleaning/kg.gpickle")